In [1]:
# class SimulationManager:
#     def __init__(self, config):
#         self.C = config
#         self.objects = []
#         self.previous_target = None

#     def setScenario(self, number_of_obstacles):
#         for i in range(number_of_obstacles):
#             # Add obstacle frames
#             parent_frame = 'parking_space_' + str(np.random.randint(1, 42))
#             if self.C.getFrame(parent_frame) is None:
#                 print(f"Error: Parent frame '{parent_frame}' does not exist.")
#                 continue

#             f = self.C.addFrame(f'car{i}', parent_frame)
#             f.setRelativePose('t(0 0 0.14) d(0 0 0 0)')
#             f.setShape(ry.ST.box, [0.9, 0.39, 0.28])
#             f.setColor([0.2, 0.2, 0.2, 0.4])
#             f.setContact(1)
#             f.setMass(1.0)

#     def getAttributes(self, object_name):
#         # Determine object type
#         if object_name.startswith('l_robot_base'):
#             obj_type = 'robot'
#         elif object_name.startswith('target_'):
#             obj_type = 'target'
#         elif object_name.startswith('car'):
#             obj_type = 'obstacle'
#         else:
#             obj_type = 'unknown'
  
#         # Retrieve the frame
#         frame = self.C.getFrame(object_name)
#         if frame is None:
#             print(f"Error: Frame for object '{object_name}' not found.")
#             return None

#         # Check if the object already exists
#         for obj in self.objects:
#             if obj['name'] == object_name:
#                 return obj

#         # Collect attributes and add to the list
#         attributes = {
#             "name": object_name,
#             "type": obj_type,
#             "frame": frame,
#             "position": frame.getPosition(),
#             "size": frame.getSize(),
#             "quaternion": frame.getQuaternion()
#         }
#         self.objects.append(attributes)
#         return attributes

#     def setTarget(self, object_name):
#         # Retrieve the frame
#         frame = self.C.getFrame(object_name)
#         if frame is None:
#             print(f"Error: Frame for object '{object_name}' not found.")
#             return None

#         # Revert previous target's color
#         if self.previous_target:
#             prev_frame = self.C.getFrame(self.previous_target)
#             if prev_frame:
#                 prev_frame.setColor([0.2, 0.2, 0.2, 0.4])

#         # Update current target's color
#         frame.setColor([0, 1, 0, 0.4])

#         # Update object type in the list
#         for obj in self.objects:
#             if obj['name'] == self.previous_target:
#                 obj['type'] = 'obstacle'
#             if obj['name'] == object_name:
#                 obj['type'] = 'target'

#         # Store the current target
#         self.previous_target = object_name

# # Initialize simulation
# C = ry.Config()
# C.addFile('/home/alperen/Desktop/cs449_project/scenarios/simEnv.g')

# # Create a SimulationManager instance
# sim_manager = SimulationManager(C)

# # Retrieve attributes for all objects
# for frame_name in C.getFrameNames():
#     sim_manager.getAttributes(frame_name)

# # Set an object as the target
# sim_manager.setTarget('car2')

# # Retrieve attributes for the target
# g = sim_manager.objects
# # View the simulation
# C.view()


In [2]:
# # Set up the scenario
# sim_manager.setScenario(5)

# # Retrieve attributes for the target
# g = sim_manager.objects
# # View the simulation
# C.view()

In [52]:
import robotic as ry
import numpy as np
import time

C = ry.Config()

In [71]:
C.addFile("simEnv2.g")

C.view()

0

In [54]:
print(C.getFrame('l_robot_base').getPosition())
print(C.getJointState())

[-3.8   0.5   0.17]
[-0.3  0.   0.   0.   0. ]


In [55]:
def moveToTarget(C, target):
    q0 = C.getJointState()

    komo = ry.KOMO(C, 5, 10, 1, False)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
    komo.addObjective([], ry.FS.positionDiff, ['l_middle_joint', target.name], ry.OT.eq, [1e1], [0.1, -0.1, 0])
    komo.addObjective([], ry.FS.scalarProductXX, ['l_robot_base', target.name], ry.OT.eq, [1e2], [-1])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ).solve()
    q = komo.getPath()[-1]

    return q

In [56]:
def moveToPark(C, target):
    q0 = C.getJointState()
    if target.getPosition()[0] > 0:
        x = -0.6
        product = 1
    else: 
        x = 0.6
        product = -1

    komo = ry.KOMO(C, 5, 10, 1, False)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
    komo.addObjective([], ry.FS.positionDiff, ['car1', target.name], ry.OT.eq, [1e0], [0, -0.1, 0.15])
    komo.addObjective([], ry.FS.scalarProductXX, ['l_robot_base', target.name], ry.OT.eq, [1e2], [product])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ).solve()
    q = komo.getPath()[-1]

    return q

In [57]:
def moveBack(C):
    q0 = C.getJointState()
    
    komo = ry.KOMO(C, 5, 10, 1, False)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
    komo.addObjective([], ry.FS.positionDiff, ['l_middle_joint', 'robot_start'], ry.OT.eq, [1e1], [0, 0.05, 0])
    komo.addObjective([], ry.FS.scalarProductXX, ['l_middle_joint', 'robot_start'], ry.OT.eq, [1e2], [1])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ).solve()
    q = komo.getPath()[-1]
    
    return q

In [58]:
car = C.getFrame('car1')
qHome = C.getJointState()
robot_to_car = moveToTarget(C, car)

In [59]:
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [robot_to_car])

for trial in range(10):
    ret = rrt.solve()
    path = ret.x
    print(ret.feasible)
    if ret.feasible is False:
        continue
    else:
        break

True-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (l_robot_body)-(car1) [97,49] 	d=-0
proxy:  (l_robot_base)-(car1) [96,49] 	d=-0



In [60]:
C.view()
time.sleep(.8)
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

In [61]:
#attach cargo
C.getFrame('car1').unLink()
C.attach('l_robot_base', 'car1') 


In [62]:
target = C.getFrame('parking_space_20')
#robotPos = C.getFrame('l_robot_base').getPosition()
robot_to_park = moveToPark(C, target)

In [63]:
rrt2 = ry.PathFinder()
rrt2.setProblem(C, [robot_to_car], [robot_to_park])

# ret2 = rrt2.solve()
# path2 = ret2.x
for trial in range(10):
    ret2 = rrt2.solve()
    path2 = ret2.x
    print(ret2.feasible)
    if ret2.feasible is False:
        continue
    else:
        break

True


In [64]:
C.view()
time.sleep(.8)
# display the path
for t in range(0, path2.shape[0]-1):
    C.setJointState(path2[t])
    C.view()
    time.sleep(.1)

In [65]:
#detach cargo
C.getFrame('car1').unLink()

In [66]:
return_back = moveBack(C)

rrt3 = ry.PathFinder()
rrt3.setProblem(C, [robot_to_park], [return_back])

ret3 = rrt3.solve()
path3 = ret3.x

-- RRT_PathFinder.cpp:RRT_PathFinder:257(0) initializing with infeasible q0:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0


In [67]:
C.view()
time.sleep(.8)
# display the path
for t in range(0, path3.shape[0]-1):
    C.setJointState(path3[t])
    C.view()
    time.sleep(.1)

In [68]:
print(C.getFrame("car2").getPosition())
print(C.getFrame("parking_space_23").getPosition())

[ 1.5  -3.6   0.15]
[-0.5  -4.2   0.01]


In [69]:
#C.setJointState(qHome)

# ry.params_add({'botsim/verbose': 2., 'physx/motorKp': 10000., 'physx/motorKd': 1000.})
# ry.params_print()
# bot = ry.BotOp(C, False)
# bot.home(C)

# bot.move(robot_to_car, [2. , 3.])
# while bot.getTimeToEnd()>0:
#     bot.sync(C, .1)

# bot.moveAutoTimed(path, 1., 1.)
# while bot.getTimeToEnd()>0:
#     bot.sync(C, .1)

# C.getFrame('car2').unLink()
# C.attach('l_robot_base', 'car2') 

# target = C.getFrame('parking_space_28')
# #robotPos = C.getFrame('l_robot_base').getPosition()
# robot_to_park = moveToPark(C, target)

# rrt2 = ry.PathFinder()
# rrt2.setProblem(C, [robot_to_car], [robot_to_park])

# for trial in range(10):
#     ret2 = rrt2.solve()
#     path2 = ret2.x
#     print(ret2.feasible)
#     if ret2.feasible is False:
#         continue
#     else:
#         break

# bot.moveAutoTimed(path2, 1., 1.)
# while bot.getTimeToEnd()>0:
#     bot.sync(C, .1)

# C.getFrame('car2').unLink()

# return_back = moveBack(C)

# rrt3 = ry.PathFinder()
# rrt3.setProblem(C, [robot_to_park], [return_back])

# ret3 = rrt3.solve()
# path3 = ret3.x

# bot.moveAutoTimed(path3, 1., 1.)
# while bot.getTimeToEnd()>0:
#     bot.sync(C, .1)


# for t in range(0, path.shape[0]-1):
#     bot.sync(C, .1)
#     bot.moveTo(path[t])

In [70]:
# del bot